In [1]:
from SammBaseLib.LogicSamm import *
from SammBaseLib.WidgetSammBase import *
import SampleData


In [2]:
# nodeToClone = SampleData.SampleDataLogic().downloadMRHead()
# nodeToClone = SampleData.SampleDataLogic().downloadCTChest()
# nodeToClone = SampleData.SampleDataLogic().downloadCTACardio()

slicer.util.selectModule("SammBase")

In [35]:
import vtk

logic = SammBaseLogic()
# logic._parameterNode.SetNodeReferenceID("sammPrompt2DBox", "vtkMRMLMarkupsPlaneNode1")
# plane = logic._parameterNode.GetNodeReference("sammPrompt2DBox")
# points = vtk.vtkPoints() 
# plane.GetPlaneCornerPoints(points)
# print(points)
# print(points.GetPoint(0)[2])

logic._parameterNode.SetNodeReferenceID("sammInputVolume", "vtkMRMLScalarVolumeNode1")
volumeNodeDataPointer = slicer.util.arrayFromVolume(
    logic._parameterNode.GetNodeReference("sammInputVolume"))
volumeDisplayNode = logic._parameterNode.GetNodeReference("sammInputVolume").GetDisplayNode()
window = volumeDisplayNode.GetWindow()
level = volumeDisplayNode.GetLevel()
print(window)
imageMin = level - window / 2
imageMax = level + window / 2
imageNormalized = (((copy.deepcopy(volumeNodeDataPointer) - imageMin).astype("float32") / (imageMax - imageMin)) * 256)
imageNormalized[imageNormalized<0] = 0
imageNormalized[imageNormalized>255] = 255
imageNormalized = imageNormalized.astype(numpy.uint8)
print(imageNormalized[127][127])

143.6402877697842
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   3   1   0   0   0   0   0   0   0   0   0   0   0   0   3
   0   0   0   0   0   0   0   0   0   0   0   0   0   3   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   8   0   0   1  32  55  56  46  42  49  56  56  53  60  74  76
  69  76  90  94  94  96 105 106 101 106  99  78  81 110 121 108 103 112
 108 103 112 119 112 112 119 121 124 128 146 151 146 160 181 187 197 219
 231 226 217 224 231 217 212 226 226 215 215 201 172 158 156 146 126 119
 117 119 130 128 101  76  55  26   5   0   3  12   3   3  12   5   0   1
   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0   0
   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0 

In [14]:
logic = SammBaseLogic()
logic._parameterNode.SetNodeReferenceID("sammInputVolume", "vtkMRMLScalarVolumeNode1")
# imageNormalized = logic.processSlicePreProcess()
# print(imageNormalized.shape)

<MRMLCore.vtkMRMLScalarVolumeNode(0xd1327f0) at 0x7f36ab2ba940>

In [34]:
# s = logic._parameterNode.GetNodeReference("sammInputVolume")

# s.SetIJKToRASDirections(numpy.array([[1,0,0],[0,1,0],[0,0,1]]))
IjkToRasDir = numpy.array([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]])
print(logic._parameterNode.GetNodeReference("sammInputVolume"))
logic._parameterNode.GetNodeReference("sammInputVolume").GetIJKToRASDirections(IjkToRasDir)
print(IjkToRasDir)

vtkMRMLScalarVolumeNode (0xcf59b90)
  ID: vtkMRMLScalarVolumeNode1
  ClassName: vtkMRMLScalarVolumeNode
  Name: DZ-CBCT_1
  Debug: false
  MTime: 1094405
  Description: (none)
  SingletonTag: (none)
  HideFromEditors: false
  Selectable: true
  Selected: false
  UndoEnabled: false
  Node references:
    display [displayNodeRef]: vtkMRMLScalarVolumeDisplayNode1
    storage [storageNodeRef]: (none)
    transform [transformNodeRef]: (none)
  TransformNodeID: (none)
  DisplayNodeIDs[0]: vtkMRMLScalarVolumeDisplayNode1
  Spacing: (0.25, 0.25, 0.25)
  Origin: (90.1896, 136.381, -49.9637)
  VoxelVectorType: undefined
  IJKToRASDirections:
    -0.994078 -0.0969441 0.0490957
    0.0913317 -0.990171 -0.105923
    0.0588818 -0.100812 0.993162
  ImageData:
    Debug: Off
    Modified Time: 1058346
    Reference Count: 4
    Registered Events: 
      Registered Observers:
        vtkObserver (0xc48f1f0)
          Event: 33
          EventName: ModifiedEvent
          Command: 0xbae2ec0
          Pr

In [31]:
inModel         = logic._parameterNode.GetNodeReference("sammInputVolume")
imageData       = slicer.util.arrayFromVolume(inModel)
imageDataShape  = imageData.shape
logic._segNumpy  = numpy.zeros(imageDataShape)

# get axis directions aligning RGY views (need to optimize here)
IjkToRasDir  = vtk.vtkMatrix4x4()
print(IjkToRasDir)
logic._parameterNode.GetNodeReference("sammInputVolume").GetRASToIJKMatrix(IjkToRasDir)
print(IjkToRasDir)


logic._parameterNode.RGYNpArrOrder = [0, 0, 0]
print(IjkToRasDir)
for i in range(3):
    logic._parameterNode.RGYNpArrOrder[i] = 2-numpy.argmax(numpy.abs(IjkToRasDir.transpose()[i]))
metadata = [ \
    [], \
    logic._parameterNode.RGYNpArrOrder
]
logic._parameterNode._volMetaData = metadata

volumeNodeDataPointer = slicer.util.arrayFromVolume(
            logic._parameterNode.GetNodeReference("sammInputVolume"))
volumeDisplayNode = logic._parameterNode.GetNodeReference("sammInputVolume").GetDisplayNode()
window = volumeDisplayNode.GetWindow()
level = volumeDisplayNode.GetLevel()

imageMin = level - window / 2
imageMax = level + window / 2
print(logic._parameterNode.RGYNpArrOrder)
imageNormalized = (((copy.deepcopy(volumeNodeDataPointer) - imageMin).astype("float32") / (imageMax - imageMin)) * 256).astype(numpy.uint8)
imageNormalized = imageNormalized.transpose(2-np.array(logic._parameterNode.RGYNpArrOrder))

vtkMatrix4x4 (0xc4c0730)
  Debug: Off
  Modified Time: 2040760
  Reference Count: 1
  Registered Events: (none)
  Elements:
    1 0 0 0 
    0 1 0 0 
    0 0 1 0 
    0 0 0 1 


vtkMatrix4x4 (0xc4c0730)
  Debug: Off
  Modified Time: 2040771
  Reference Count: 1
  Registered Events: (none)
  Elements:
    0 -1 0 133.929 
    -0 0 -1 116.786 
    0.769233 0 0 66.6502 
    -0 0 -0 1 


vtkMatrix4x4 (0xc4c0730)
  Debug: Off
  Modified Time: 2040771
  Reference Count: 1
  Registered Events: (none)
  Elements:
    0 -1 0 133.929 
    -0 0 -1 116.786 
    0.769233 0 0 66.6502 
    -0 0 -0 1 




<class 'AttributeError'>: 'vtkmodules.vtkCommonMath.vtkMatrix4x4' object has no attribute 'transpose'

In [4]:
print(logic._parameterNode.GetParameter("sammViewOptions")=="RED")

True


In [5]:
s = slicer.app.layoutManager().sliceWidget('Red').mrmlSliceNode()
mat = s.GetXYToRAS()
temp = mat.MultiplyPoint([0,0,0,1])
print(temp)
ras2ijk = vtk.vtkMatrix4x4()
logic._parameterNode.GetNodeReference("sammInputVolume").GetRASToIJKMatrix(ras2ijk)
temp = ras2ijk.MultiplyPoint(temp)
print(temp)
print(logic._parameterNode.RGYNpArrOrder)
temp = np.array([temp[0],temp[1],temp[2]])[logic._parameterNode.RGYNpArrOrder]
print(temp)

(129.72245434031765, -121.57139587402344, -10.214302062988281, 1.0)
(2.4310518326593353, 7.548171374262596, 4.936334692913553, 1.0)
[2, 1, 0]
[4.93633469 7.54817137 2.43105183]


In [6]:
volumeNodeDataPointer = slicer.util.arrayFromVolume(
    logic._parameterNode.GetNodeReference("sammInputVolume"))
volumeDisplayNode = logic._parameterNode.GetNodeReference("sammInputVolume").GetDisplayNode()
window = volumeDisplayNode.GetWindow()
level = volumeDisplayNode.GetLevel()

imageMin = level - window / 2
imageMax = level + window / 2

imageNormalized = (((copy.deepcopy(volumeNodeDataPointer) - imageMin).astype("float32") / (imageMax - imageMin)) * 256).astype(numpy.uint8)
print(imageNormalized.shape)
imageNormalized = imageNormalized.transpose(2-np.array(logic._parameterNode.RGYNpArrOrder))
print(imageNormalized.shape)

(10, 10, 10)
(10, 10, 10)


In [10]:
curslc = logic.utilGetCurrentSliceIndex()
print(curslc)

127.0


In [12]:
logic.processInitPromptSync()
prompt_add_point, prompt_remove_point = [], []
curslc = logic.utilGetCurrentSliceIndex()
numControlPoints = logic._prompt_add.GetNumberOfControlPoints()
for i in range(numControlPoints):
    ras = vtk.vtkVector3d(0,0,0)
    logic._prompt_add.GetNthControlPointPosition(i,ras)
    temp = logic._volumeRasToIjk.MultiplyPoint([ras[0],ras[1],ras[2],1])
    temp = np.array([temp[0], temp[1], temp[2]])[logic._parameterNode.RGYNpArrOrder]
    prompt_add_point.append(logic.utilGetPositionOnSlicer(temp))

numControlPoints = logic._prompt_remove.GetNumberOfControlPoints()
for i in range(numControlPoints):
    ras = vtk.vtkVector3d(0,0,0)
    logic._prompt_remove.GetNthControlPointPosition(i,ras)
    temp = logic._volumeRasToIjk.MultiplyPoint([ras[0],ras[1],ras[2],1])
    temp = np.array([temp[0], temp[1], temp[2]])[logic._parameterNode.RGYNpArrOrder]
    prompt_remove_point.append(logic.utilGetPositionOnSlicer(temp))

print(prompt_add_point)
print(prompt_remove_point)

[[89, 75], [123, 102], [152, 66]]
[[132, 35], [192, 55], [180, 95]]
[SAMM INFO] Sent Size Command.


100%|##########| 256/256 [00:00<00:00, 868.94it/s]


[SAMM INFO] Sent Image.
[SAMM INFO] Sent Embedding Computing Command.


In [13]:
logic.processInitPromptSync()
logic.processStartPromptSync()

In [7]:
logic._parameterNode.GetParameter("sammViewOptions")[0]

'R'

[SAMM INFO] Sent Size Command.


100%|##########| 10/10 [00:00<00:00, 914.07it/s]


[SAMM INFO] Sent Image.
[SAMM INFO] Sent Embedding Computing Command.


/home/yl/Slicer/bin/Python/slicer/util.py:2521: UserWarning: does not have observer
  warn('does not have observer')


[SAMM INFO] Sent Size Command.


100%|##########| 10/10 [00:00<00:00, 895.15it/s]


[SAMM INFO] Sent Image.
[SAMM INFO] Sent Embedding Computing Command.


Traceback (most recent call last):
  File "/home/yl/software/samm/samm/SammBase/SammBaseLib/WidgetSammBase.py", line 162, in onPushStartMaskSync
    self.logic.processStartPromptSync()
  File "/home/yl/software/samm/samm/SammBase/SammBaseLib/LogicSamm.py", line 293, in processStartPromptSync
    self._connections.pushRequest(SammMsgType.INFERENCE, {
  File "/home/yl/software/samm/samm/SammBase/SammBaseLib/UtilConnections.py", line 40, in pushRequest
    feedback = sock.recv()
  File "zmq/backend/cython/socket.pyx", line 805, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 841, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 199, in zmq.backend.cython.socket._recv_copy
  File "zmq/backend/cython/socket.pyx", line 194, in zmq.backend.cython.socket._recv_copy
  File "zmq/backend/cython/checkrc.pxd", line 22, in zmq.backend.cython.checkrc._check_rc
zmq.error.Again: Resource temporarily unavailable

If you suspect t

In [24]:
import numpy as np

x = np.array([0.1, 0.2, 0.3])
i = np.array([1,0,2])
y = x[i]
print(y)
ii = i.argsort()
z = y[ii]
print(z)

[0.2 0.1 0.3]
[0.1 0.2 0.3]


In [40]:
logic._parameterNode.RGYNpArrOrder
x = np.array([0.1, 0.2, 0.3])
y = x[2-np.array(logic._parameterNode.RGYNpArrOrder)]
ii = np.array(2-np.array(logic._parameterNode.RGYNpArrOrder)).argsort()
z = y[ii]
z = np.zeros((1,2,3))
print(z.reshape((2,3,1)))

[[[0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]]]


In [6]:
import numpy as np
np.array([1,2,3,4]).reshape([1,4])[0]

array([1, 2, 3, 4])

In [20]:
print([]==None)

False


In [3]:
import numpy as np
a = np.array([[1,2],[3,4]])
print(np.amax(a))

4
